In [1]:
import json # a module that provides methods for parsing json

In [2]:
import time # a module that provides various time related functions

In [3]:
import tiktoken #tokenization tool for GPT series models 

In [4]:
import jsonlines

In [5]:
from openai import OpenAI #client lib that enables authentication and making requests 

In [6]:
OpenAI_API_KEY = open('OPENAI_API_KEY.txt').read().strip()

In [7]:
client = OpenAI(
    api_key = OpenAI_API_KEY
)

In [8]:
with jsonlines.open("C:\\Users\\Ghassan\\Desktop\\haya jupyter notebook\\Scholarship Search\\scholarship_data_2.json" , "r") as f:
    all_content= list(f.iter())

print(len(all_content))

10


In [9]:
only_content = []
for item in all_content:
    only_content.append(item['content'])

In [10]:
for content in only_content:
    print(content)
    print(100*'#')

Fairleigh Dickinson Scholarships for International Students (link: https://www.scholars4dev.com/25896/fairleigh-dickinson-scholarships-for-international-students/)
Read More (link: https://www.scholars4dev.com/25896/fairleigh-dickinson-scholarships-for-international-students/)
Fulbright Foreign Student Program in USA (link: https://www.scholars4dev.com/2876/usa-fulbright-scholarships-for-international-students/)
Read More (link: https://www.scholars4dev.com/2876/usa-fulbright-scholarships-for-international-students/)
Fully-Funded Rotary Peace Fellowships (link: https://www.scholars4dev.com/8088/rotary-peace-fellowships/)
Read More (link: https://www.scholars4dev.com/8088/rotary-peace-fellowships/)
Illinois Wesleyan University International Students Scholarships (link: https://www.scholars4dev.com/25841/illinois-wesleyan-university-international-students-scholarships/)
Read More (link: https://www.scholars4dev.com/25841/illinois-wesleyan-university-international-students-scholarships/)


In [11]:
print(len(all_content))

10


In [12]:
together = 0
for content in only_content:
    content_length = len(content)
    print(f"Content length: {content_length}")
    together = together + content_length

print("the length of all the content together is: " , together)

Content length: 65755
Content length: 198581
Content length: 100203
Content length: 167484
Content length: 56951
Content length: 28063
Content length: 66275
Content length: 149
Content length: 112882
Content length: 61199
the length of all the content together is:  857542


In [13]:
def num_of_tokens(text):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(tokenizer.encode(text))

content_with_tokens = [num_of_tokens(content) for content in only_content]

content_with_tokens

[17863, 50812, 26957, 39257, 19200, 7768, 17211, 32, 27813, 17408]

In [14]:
def split_segments(content, window_size, step_size): #function to split the content into segments 
    words = content.split()
    segments = [] # initialize an empty list 
    for start in range(0, len(words), step_size):
        # Create a segment from the current start position to start + window_size
        segment = words[start:start + window_size] 
        segments.append(' '.join(segment)) #join words in the segment into a single string with spaces between the words and appends this string to the segment list
        print(f"Segment created from word {start} to {start + window_size}")
    return segments

In [23]:
WINDOW_SIZE = 3000  
STEP_SIZE = 1500    
TPM_LIMIT = 60000 #token per min
RPM_LIMIT = 3500 #request per min
CONTEXT_LIMIT = 16385 #token per single api call

tokens_used = 0
requests_made = 0

responses = []
final_response=[]

all_segments= []
all_tokens = []

In [16]:
question = "extract information about all the scholarships mentioned put every scholarship u find with its information in a json format"

In [17]:
question2 = "extract information about all the scholarships mentioned put every scholarship u find with its information in a  this json format Scholarship Name,Scholarship link, Provider,Degree Level,Deadline,Study Location,Course Start,last_updated,status,frequency,next_course_starts,Funding,Eligibility,Eligibility Criteria,Eligibility Country,Eligibility GPA Scale,Coverage, Award, the one that u dont find put not applicable"

In [18]:
def generate_responses(question , content):
    response = client.chat.completions.create( #invokes the openai api to generate a competion
        model="gpt-3.5-turbo",
        messages=[
            # Provide a system message to set the context for the assistant
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"{question2}\nText: {content}"}
        ]
    )
    return response.choices[0].message.content.strip()# Return the assistant's response, removing any leading or trailing whitespace

In [19]:
print(len(all_segments))

0


In [24]:
start_time = time.time()
for content in only_content:
    segments = split_segments(content, WINDOW_SIZE, STEP_SIZE)
    all_segments.extend(segments)
    
    for segment in segments:
        tokens = num_of_tokens(segment)
        
        # Split segment if it exceeds context limit
        if tokens > CONTEXT_LIMIT:
            # Create sub-segments of size `CONTEXT_LIMIT`
            sub_segments = [segment[i:i + CONTEXT_LIMIT] for i in range(0, len(segment), CONTEXT_LIMIT)]
        else:
            sub_segments = [segment]

        for sub_segment in sub_segments:
            tokens = num_of_tokens(sub_segment)
            # Check if adding this sub-segment would exceed the token or request limits
            if tokens_used + tokens > TPM_LIMIT or requests_made + 1 > RPM_LIMIT:
                elapsed_time = time.time() - start_time
                if elapsed_time < 60:
                    time.sleep(60 - elapsed_time)
                tokens_used = 0
                requests_made = 0
                start_time = time.time()

            response = generate_responses(question2, sub_segment)
            final_response.append(response)
            tokens_used += tokens
            requests_made += 1

# Save final responses to JSON file
output_file = 'response_final.json'
with open(output_file, 'w') as f:
    json.dump(final_response, f, indent=4)

print(f'Responses saved to {output_file}')

Segment created from word 0 to 3000
Segment created from word 1500 to 4500
Segment created from word 3000 to 6000
Segment created from word 0 to 3000
Segment created from word 1500 to 4500
Segment created from word 3000 to 6000
Segment created from word 4500 to 7500
Segment created from word 6000 to 9000
Segment created from word 7500 to 10500
Segment created from word 9000 to 12000
Segment created from word 10500 to 13500
Segment created from word 12000 to 15000
Segment created from word 13500 to 16500
Segment created from word 0 to 3000
Segment created from word 1500 to 4500
Segment created from word 3000 to 6000
Segment created from word 4500 to 7500
Segment created from word 0 to 3000
Segment created from word 1500 to 4500
Segment created from word 3000 to 6000
Segment created from word 4500 to 7500
Segment created from word 6000 to 9000
Segment created from word 7500 to 10500
Segment created from word 9000 to 12000
Segment created from word 10500 to 13500
Segment created from word

In [25]:
for res in final_response:
    print(res)

```json
{
    "scholarships": [
        {
            "Scholarship Name": "Fairleigh Dickinson Scholarships for International Students",
            "Scholarship link": "https://www.scholars4dev.com/25896/fairleigh-dickinson-scholarships-for-international-students/",
            "Provider": "Fairleigh Dickinson University",
            "Degree Level": "Bachelors",
            "Deadline": "1 July/1 Dec 2024 (Annual)",
            "Study Location": "USA",
            "Course Start": "Fall 2024/Spring 2025",
            "last_updated": "16 Apr 2024",
            "status": "CURRENTLY CLOSED",
            "frequency": "Annual",
            "next_course_starts": "Fall 2024/Spring 2025",
            "Funding": "Not applicable",
            "Eligibility": "Not applicable",
            "Eligibility Criteria": "Not applicable",
            "Eligibility Country": "Not applicable",
            "Eligibility GPA Scale": "Not applicable",
            "Coverage": "Not applicable",
            "Award":

In [26]:
print(len(final_response))

56
